In [11]:
"""
Error Code:
    0: Error opening file
"""

import pandas as pd
import numpy as np
import zipfile

# Clarify the mission N number for this .ipynb
N_number = 1

In [3]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def display(items):
    print(namestr(items, globals()))
    for item in items:
        print("    " + item)

# Test display
test_dict = {"A": [1, 2, 3], "B": [4, 5, 6]}
display(test_dict)

['test_dict']
    A
    B


In [4]:
from os import listdir
from os.path import isfile, join

mypath = "./"
zip_file_name = "journals"
zip_file = zipfile.ZipFile(mypath + zip_file_name + ".zip")
file_name_list = zip_file.namelist()
# Should return 4 folders if testing on ASG Zip
print("{0} files detected under current directory. ".format(len(file_name_list)))

40 files detected under current directory. 


In [5]:
def filter_by_filename(files_list):
    """Function filtering files by filenames.
    Only accepts files starts with "journal-article"
    """
    filtered_list = []
    mask_match = zip_file_name + "/ngram" + str(N_number) + "/journal-article"
    for filename in files_list:
        # Check if the filename starts with "journal-article"
        assert isinstance(filename, str)
        # Check the first 20 characters of the file name
        if filename.startswith(mask_match, 0, len(mask_match)):
            filtered_list.append(filename)
    return filtered_list

# Run "filter_by_filename" for current directory
filtered_list = filter_by_filename(file_name_list)
print("{0} files remained after filename filtering. ".format(len(filtered_list)))

10 files remained after filename filtering. 


In [6]:
# Functions checking word attributes (single-letter, starts/ends with numebr)
def is_single_letter(word):
    assert isinstance(word, str)
    return len(word) <= 1

def starts_with_number(word):
    assert isinstance(word, str)
    try:
        return word[0].isdigit()
    except:
        return False

def ends_with_number(word):
    assert isinstance(word, str)
    try:
        return word[len(word) - 1].isdigit()
    except:
        return False

# Summary of check functions
check_funcs = [
    is_single_letter, 
    starts_with_number, 
    ends_with_number,
]

In [7]:
file_0 = filtered_list[0]
try:
    journal_0 = zip_file.open(file_0, mode="r")
except IOError:
    print("Error opening file {0}".format(file_0))
    exit(0)

In [8]:
# Track the meaningful dictonaries
freq_dict = {}

# Read file line by line
for line in journal_0:
    line = line.decode("ascii") 
    assert isinstance(line, str)
    pair = line.strip().split()
    
    # Separate word/freq
    word, freq = pair
    assert isinstance(word, str)
    
    # Filter by word's attribute
    check_results = [check_func(word) for check_func in check_funcs]
    if any(check_results):
        continue
    freq_dict.update({word: freq})

journal_0.close()
display(freq_dict)

['freq_dict']
    pp
    paper
    cloth
    price
    listed
    institute
    research
    ed
    new
    policy
    washington
    ma
    public
    york
    american
    enterprise
    co
    social
    employment
    press
    publications
    publishing
    ca
    cambridge
    danish
    housing
    labor
    university
    douglas
    economic
    economics
    health
    human
    london
    problems
    report
    united
    welfare
    work
    academic
    adams
    allan
    allowance
    approach
    assistance
    ballinger
    care
    commission
    committee
    community
    consequences
    copenhagen
    corporation
    current
    effects
    english
    experiment
    final
    handbook
    immigration
    inc
    international
    issues
    jerry
    journal
    kalamazoo
    mark
    market
    markets
    massachusetts
    mi
    monica
    national
    neal
    neale
    participation
    phillip
    politics
    prodist
    program
    progress
    rand
   

In [9]:
# Create Dataframe
# Data
data = []
for word in freq_dict:
    freq = freq_dict[word]
    data.append([word, freq])

# Columns
columns = ["word", "freq"]

# Index
index = list(range(len(freq_dict)))

# DataFrame
dataframe = pd.DataFrame(data, columns=columns, index=index)

# Print Test
dataframe.head()

,word,freq
0,pp,39
1,paper,27
2,cloth,17
3,price,17
4,listed,16


In [10]:
# Run filtering for each file
for file in filtered_list:
    # Open file
    try:
        file_open = zip_file.open(file, mode="r")
    except IOError:
        print("Error opening file {0}".format(file))
        exit(0)
    
    # Create output file
    output_name = "Results_1" + file[len(zip_file_name) + len("/ngram" + str(N_number)):]
    
    print(output_name)
    
    # Note: Output file encoding set to UTF-8
    output_file = open(output_name, mode="w+", encoding="utf-8")
    
    # Track for max length word
    # Used for formatting output data
    max_length = float("-inf")
        
    # Initiate freq_dict -> {word: freq}
    freq_dict = {}
    
    # Read by line
    for line in file_open:
        # Line decode using UTF-8
        line = line.decode("utf-8")
        assert isinstance(line, str)
        pair = line.strip().split()
    
        # Separate word/freq
        word, freq = pair
        assert isinstance(word, str)
        
        # Filter by word's attribute
        check_results = [check_func(word) for check_func in check_funcs]
        if any(check_results):
            continue
            
        # Update max_length
        if len(word) > max_length:
            max_length = len(word)
        freq_dict.update({word: freq})
    
    # Close reading file
    file_open.close()
    
    # Create Dataframe
    # Data
    data = []
    for word in freq_dict:
        freq = freq_dict[word]
        data.append([word, freq])
    
    # Columns
    columns = ["word", "freq"]
    
    # Index
    index = list(range(len(freq_dict)))
    
    # DataFrame
    dataframe = pd.DataFrame(data, columns=columns, index=index)
    
    # Write to output file
    for word in freq_dict:
        freq = freq_dict[word]
        output_file.write(word.ljust(max_length + 5))
        output_file.write(freq + "\n")
    
    output_file.close()
    print(dataframe.head())

    

Results_1/journal-article-10.2307_145213-ngram1.txt
     word freq
0      pp   39
1   paper   27
2   cloth   17
3   price   17
4  listed   16
Results_1/journal-article-10.2307_145224-ngram1.txt
      word freq
0   search  112
1      job   55
2      ojs   51
3     wage   43
4  workers   35
Results_1/journal-article-10.2307_145221-ngram1.txt
           word freq
0      earnings   63
1   achievement   37
2    motivation   35
3    background   34
4  intelligence   33
Results_1/journal-article-10.2307_145216-ngram1.txt
       word freq
0  teachers  160
1    school   85
2   teacher   72
3   schools   53
4  transfer   42
Results_1/journal-article-10.2307_145226-ngram1.txt
       word freq
0  earnings   41
1   college   20
2  security   18
3    social   18
4    school   16
Results_1/journal-article-10.2307_145211-ngram1.txt
       word freq
0        ui   64
1    search   48
2       job   28
3  benefits   25
4      data   15
Results_1/journal-article-10.2307_145208-ngram1.txt
         word freq